In [2]:
import spacy
nlp = spacy.load("en_core_web_md")

In [3]:
from spacy.tokenizer import Tokenizer

dummy_tokenizer = Tokenizer(nlp.vocab)

def nlp_no_tokenize(text):
    tokenizer = nlp.tokenizer
    nlp.tokenizer = dummy_tokenizer
    model = nlp(text)
    nlp.tokenizer = tokenizer
    return model

for tok in nlp_no_tokenize("dog's are funny animal's"):
    print("{} ({}) - {} - {}".format(tok.text, tok.pos_, tok.dep_, tok.head.text))

dog's (NOUN) - nsubj - are
are (AUX) - ROOT - are
funny (ADJ) - acomp - are
animal's (PUNCT) - punct - are


### Готуємо тестувальний датасет. Дані будуть складатись з Spacy токенів. Це дозволить нам визначити межі речень без додаткового маркування, а також у них міститиметься інша корисна інформація (POS теги, залежності і т.д.)

In [4]:
import json

with open('../../../tasks/06-language-as-sequence/run-on-test.json', 'r') as f:
    run_on_test = json.load(f)

In [6]:
import collections

def prepare_test_data(run_on_test):
    X = []
    y = []
    for sent in run_on_test:
        words = [word for word, _ in sent]
        labels = [label for _, label in sent]
        
        model = nlp_no_tokenize(' '.join(words))
        if (not len(sent) == len(model) or 
                any(not tok.text == word for tok, word in zip(model, words))):
            print('WARN: bad model: ', model)
        X.extend(model)
        y.extend(labels)
    return X, y

X_test, y_test = prepare_test_data(run_on_test)
collections.Counter(y_test)

Counter({False: 4542, True: 155})

### Мінімалістичний бейзлайн на одному правилі :)

In [7]:
def baseline_classify_token(tok):
    next = tok.nbor() if tok.i < len(tok.doc) - 1 else None    
    return (not next is None) and next.text.istitle() and not tok.text.istitle()

def baseline_classify(X):
    return [baseline_classify_token(x) for x in X]

In [8]:
from sklearn.metrics import classification_report
print(classification_report(y_test, baseline_classify(X_test)))

              precision    recall  f1-score   support

       False       0.98      0.96      0.97      4542
        True       0.29      0.44      0.35       155

    accuracy                           0.95      4697
   macro avg       0.64      0.70      0.66      4697
weighted avg       0.96      0.95      0.95      4697



### Підготовка тренувальних даних. Для цього використав корпус Брауна.

In [9]:
from nltk.corpus import brown
brown_sents = list(brown.sents())
brown_sents[0]

['The',
 'Fulton',
 'County',
 'Grand',
 'Jury',
 'said',
 'Friday',
 'an',
 'investigation',
 'of',
 "Atlanta's",
 'recent',
 'primary',
 'election',
 'produced',
 '``',
 'no',
 'evidence',
 "''",
 'that',
 'any',
 'irregularities',
 'took',
 'place',
 '.']

In [10]:
import random
import collections

def num_sents_to_glue():
    x = random.uniform(0, 1)
    # we need:
    #  - 60% of two glued sentences,
    #  - 25% of three glued sentences,
    #  - 15% of four glued sentences.
    if x < 0.15:
        return 4
    elif x >= 0.15 and x < 0.4:
        return 3
    else:
        return 2

def need_lowercase():
    return random.uniform(0, 1) > 0.5

print(collections.Counter(map(lambda x: need_lowercase(), range(1000))))
print(collections.Counter(map(lambda x: num_sents_to_glue(), range(1000))))

Counter({False: 505, True: 495})
Counter({2: 596, 3: 241, 4: 163})


In [18]:
def glue_and_mark(sents):
    labels = []
    
    # while brown corpus is already tokenized,
    # we still need to reparse it with Spacy, since it uses
    # different tokenization scheme (e.g. in nltk 
    # "don't" is one token, but in Spacy it's two tokens)
    joined_sents = [' '.join(s) for s in sents]
    sent_models = [nlp(s) for s in joined_sents]
    
    words = []
    labels = []
    
    def do_glue(model, strip_punct, do_lowercase):
        model = model[0:-1] if strip_punct and model[-1].is_punct else model
        
        for tok in model:
            lowercasing = (tok.i == 0 and do_lowercase and need_lowercase())
            text = tok.lower_ if lowercasing else tok.text
            is_break = (strip_punct and tok.i == (len(model) - 1))
            words.append(text)
            labels.append(is_break)
    
    do_glue(sent_models[0], True, False)
    for s in sent_models[1:-1]:
        do_glue(s, True, True)
    do_glue(sent_models[-1], False, True)
    
    # join sentences with dropped punctuation and do parse again.
    # also check whether everything is OK, after parse
    glued_model = nlp_no_tokenize(' '.join(words))
    if (not len(glued_model) == len(words) or 
            any([not tok.text == word for tok, word in zip(glued_model, words)])):
        println('WARN: bad glue')
        return ([], [])
    
    return glued_model, labels
    
    
def prepare_train_data(sents):
    quote_re = re.compile('[\'`]')
    
    # drop sentences that start or end with quotes
    sents = [s for s in sents if (not re.search(quote_re, s[0]) and 
                                  not re.search(quote_re, s[-1]))]
    i = 0
    X = []
    y = []
    while i < len(sents):
        n = num_sents_to_glue()
        to_glue = sents[i:i+n]
        if len(to_glue) > 1:
            tokens, labels = glue_and_mark(to_glue)
            X.extend(tokens)
            y.extend(labels)
        i += n
    return X, y

In [19]:
X_train, y_train = prepare_train_data(brown_sents)
collections.Counter(y_train)

Counter({False: 1049362, True: 31510})

In [24]:
for x, y in zip(X_train[:30], y_train[:30]):
    print("{} - {}".format(x.text, y))

The - False
Fulton - False
County - False
Grand - False
Jury - False
said - False
Friday - False
an - False
investigation - False
of - False
Atlanta - False
's - False
recent - False
primary - False
election - False
produced - False
` - False
` - False
no - False
evidence - False
'' - False
that - False
any - False
irregularities - False
took - False
place - True
the - False
jury - False
further - False
said - False


### Утиліти для створення класифікатору та крос-валідації

In [25]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, recall_score, f1_score, precision_score

cv_scoring = {'recall_True': make_scorer(recall_score, average = None, labels = [True]), 
              'recall_False': make_scorer(recall_score, average = None, labels = [False]),
              'precision_True': make_scorer(precision_score, average = None, labels = [True]),
              'precision_False': make_scorer(precision_score, average = None, labels = [False]),
              'f1_True': make_scorer(f1_score, average = None, labels = [True]),
              'f1_False': make_scorer(f1_score, average = None, labels = [False])
             }

def cross_validation_report(clf, X, y):
    results = cross_validate(clf, X, y, scoring=cv_scoring)
    
    def calc(arr):
        mean = arr.mean()
        dev= arr.std() * 2
        
        return "%0.2f (+/- %0.2f)" % (mean, dev)
    
    print("True")
    print("\tprecision:\t{}".format(calc(results['test_precision_True'])))
    print("\trecall:\t\t{}".format(calc(results['test_recall_True'])))
    print("\tf1:\t\t{}".format(calc(results['test_f1_True'])))
    print('')
    print("False")
    print("\tprecision:\t{}".format(calc(results['test_precision_False'])))
    print("\trecall:\t\t{}".format(calc(results['test_recall_False'])))
    print("\tf1:\t\t{}".format(calc(results['test_f1_False'])))

In [187]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

def ds_func(f):
    return lambda X: [f(x) for x in X]

def combine_extractors(funcs):
    def combined(x):
        feats = {}
        for e in funcs:
            feats.update(e(x))
        return feats
    return combined

def make_classifier(*feature_extractors):
    classifier = Pipeline([('extractor', FunctionTransformer()),
                           ('dict_vect', DictVectorizer()),
                           ('lrc', LogisticRegression())])
        
    params = {'lrc__random_state': 42,
              'lrc__solver': 'sag',
              'lrc__multi_class': 'multinomial',
              'lrc__max_iter': 1000,
              'extractor__func': ds_func(combine_extractors(feature_extractors))}
    classifier.set_params(**params)

    return classifier

### Перша версія класифікатору на лог. регресії. Використовую лише слово та його контекст.

In [188]:
def word_features(tok):
    feats = {}

    feats['word'] = tok.text
    feats['word-2'] = tok.nbor(-2).text if tok.i >= 2 else '<S>'
    feats['word-1'] = tok.nbor(-1).text if tok.i >= 1 else '<S>'
    feats['word+1'] = tok.nbor(1).text if tok.i <= len(tok.doc) - 2 else '</S>'
    feats['word+2'] = tok.nbor(2).text if tok.i <= len(tok.doc) - 3 else '</S>'

    return feats

In [189]:
clf = make_classifier(word_features)
cross_validation_report(clf, X_train, y_train)

True
	precision:	0.87 (+/- 0.03)
	recall:		0.50 (+/- 0.05)
	f1:		0.64 (+/- 0.04)

False
	precision:	0.99 (+/- 0.00)
	recall:		1.00 (+/- 0.00)
	f1:		0.99 (+/- 0.00)


### Додаємо до фіч частину мови слова, а також частини мов слів в околі слова.

In [190]:
def pos_features(tok):
    feats = {}
    
    feats['pos'] = tok.tag_
    feats['pos-2'] = tok.nbor(-2).tag_ if tok.i >= 2 else '<S>'
    feats['pos-1'] = tok.nbor(-1).tag_ if tok.i >= 1 else '<S>'
    feats['pos+1'] = tok.nbor(1).tag_ if tok.i <= len(tok.doc) - 2 else '</S>'
    feats['pos+2'] = tok.nbor(2).tag_ if tok.i <= len(tok.doc) - 3 else '</S>'

    return feats

In [191]:
clf = make_classifier(word_features, pos_features)
cross_validation_report(clf, X_train, y_train)

True
	precision:	0.85 (+/- 0.03)
	recall:		0.54 (+/- 0.05)
	f1:		0.66 (+/- 0.04)

False
	precision:	0.99 (+/- 0.00)
	recall:		1.00 (+/- 0.00)
	f1:		0.99 (+/- 0.00)


### Лемма слова та лемми слів в околі

In [192]:
def lemma_features(tok):
    feats = {}

    feats['lemma'] = tok.lemma_
    feats['lemma-2'] = tok.nbor(-2).lemma_ if tok.i >= 2 else '<S>'
    feats['lemma-1'] = tok.nbor(-1).lemma_ if tok.i >= 1 else '<S>'    
    feats['lemma+1'] = tok.nbor(1).lemma_ if tok.i <= len(tok.doc) - 2 else '</S>'
    feats['lemma+2'] = tok.nbor(2).lemma_ if tok.i <= len(tok.doc) - 3 else '</S>'

    return feats

In [193]:
clf = make_classifier(word_features, pos_features, lemma_features)
cross_validation_report(clf, X_train, y_train)

True
	precision:	0.83 (+/- 0.03)
	recall:		0.55 (+/- 0.05)
	f1:		0.66 (+/- 0.04)

False
	precision:	0.99 (+/- 0.00)
	recall:		1.00 (+/- 0.00)
	f1:		0.99 (+/- 0.00)


### Залежності слова

In [156]:
def token_depth(tok, depth=0):
    if tok.dep_ == 'ROOT':
        return depth
    else:
        depth = depth+1
        if depth >= 500:
            return depth
        return token_depth(tok.head, depth) 

In [159]:
token_depth(nlp('I like cats very much')[3])

2

In [194]:
def dep_features(tok):
    feats = {}
    
    feats['dep'] = tok.dep_
    feats['dep-depth'] = token_depth(tok)
    feats['head-word'] = tok.head.text
    feats['head-lemma'] = tok.head.lemma_
    feats['head-pos'] = tok.head.pos_
        
    return feats

In [195]:
clf = make_classifier(word_features, pos_features, dep_features)
cross_validation_report(clf, X_train, y_train)

True
	precision:	0.84 (+/- 0.02)
	recall:		0.56 (+/- 0.05)
	f1:		0.67 (+/- 0.04)

False
	precision:	0.99 (+/- 0.00)
	recall:		1.00 (+/- 0.00)
	f1:		0.99 (+/- 0.00)


### Інформація про "дітей" в дереві залежностей

In [196]:
def dep_children_features(tok):
    feats = {}
    children = list(tok.children)
    feats['children-num'] = len(children)
    feats['child-1-word'] = children[0].text if len(children) > 0 else '<<<none>>>'
    feats['child-2-word'] = children[1].text if len(children) > 1 else '<<<none>>>'
    feats['child-3-word'] = children[2].text if len(children) > 2 else '<<<none>>>'
    feats['child-1-dep'] = children[0].dep_ if len(children) > 0 else '<<<none>>>'
    feats['child-2-dep'] = children[1].dep_ if len(children) > 1 else '<<<none>>>'
    feats['child-3-dep'] = children[2].dep_ if len(children) > 2 else '<<<none>>>'
    feats['child-1-lemma'] = children[0].lemma_ if len(children) > 0 else '<<<none>>>'
    feats['child-2-lemma'] = children[1].lemma_ if len(children) > 1 else '<<<none>>>'
    feats['child-3-lemma'] = children[2].lemma_ if len(children) > 2 else '<<<none>>>'
    feats['child-1-pos'] = children[0].pos_ if len(children) > 0 else '<<<none>>>'
    feats['child-2-pos'] = children[1].pos_ if len(children) > 1 else '<<<none>>>'
    feats['child-3-pos'] = children[2].pos_ if len(children) > 2 else '<<<none>>>'
    
    return feats

In [197]:
clf = make_classifier(word_features, pos_features, dep_features, dep_children_features)
cross_validation_report(clf, X_train, y_train)

True
	precision:	0.83 (+/- 0.02)
	recall:		0.57 (+/- 0.05)
	f1:		0.68 (+/- 0.03)

False
	precision:	0.99 (+/- 0.00)
	recall:		1.00 (+/- 0.00)
	f1:		0.99 (+/- 0.00)


### Збираю біграми через phrasefinder.io

In [219]:
from phrasefinder import phrasefinder as pf

def fetch_bigram(text):
#     print('fetching...')
    try:
        query = pf.escape_query_term(text)
        result = pf.search(pf.Corpus.AMERICAN_ENGLISH, query)
        if result.error:
            print('WARN: request failed: {}'.format(result.error['message']))
            return None

        return [phrase.match_count for phrase in result.phrases]
    except Exception as error:
        print('Fatal error: {}'.format(error))
        return None

    
fetch_bigram("i like")    

[4654418, 47034, 14779, 14337, 2166, 269, 123]

In [220]:
def process_bigram(bigram, res_dict):
    word1, word2 = bigram
    
    def fetch_and_save(text):
        if not text in res_dict:
            freq = fetch_bigram(text)
            if freq is not None:
                res_dict[text] = freq
    
    formatted = '{} {}'.format(word1.lower(), word2.lower())
    formatted_w_period = '{} .'.format(word1.lower())
    formatted_w_qmark = '{} ?'.format(word1.lower())
    formatted_w_excl_mark = '{} !'.format(word1.lower())

    fetch_and_save(formatted)
    fetch_and_save(formatted_w_period)
    fetch_and_save(formatted_w_qmark)
    fetch_and_save(formatted_w_excl_mark)
            
    return res_dict
        
process_bigram(['i', 'like'], {})

{'i like': [4654418, 47034, 14779, 14337, 2166, 269, 123],
 'i .': [786752, 478533, 148],
 'i ?': [1766871, 206615],
 'i !': [67902, 24809]}

In [221]:
def collect_bigrams(words, res_dict):
    print('starting...')
    bigrams = [words[i:i+2] for i in range(len(words) - 1)]
    
    for bigram in bigrams:
        process_bigram(bigram, res_dict)
    
    print('done!')
    
    return res_dict

In [222]:
bigrams_dict = {}
collect_bigrams([tok.text for tok in X_train], bigrams_dict)
collect_bigrams([tok.text for tok in X_test], bigrams_dict)


with open('bigrams.json', 'w') as f:
    json.dump(bigrams_dict, f)

starting...
done!
starting...
done!


In [223]:
def bigram_freq(word1, word2):
    arr = bigrams_dict["{} {}".format(word1.lower(), word2.lower())]
    if arr:
        return arr[0]
    else:
        return 0
bigram_freq('cats', 'are')

104819

### Тестуємо додавання у фічі різницю частот біграм '<слово> <наступне слово>' та '<слово> <крапка>'. Результат сильно погіршився :( Наскільки розумію, це через те, що ці числа дуже сильно різняться і їх важко за-fit-ити лінійною моделлю.

In [224]:
def bigrams_features(tok):
    feats = {}
    next_tok = tok.nbor(1) if tok.i <= len(tok.doc) - 2 else None
    freq = bigram_freq(tok.text, next_tok.text) if next_tok else 0
    period_freq = bigram_freq(tok.text, '.') if next_tok else 0
    feats['period-noperiod'] = period_freq - freq
    
    return feats

In [226]:
clf = make_classifier(word_features, bigrams_features)
cross_validation_report(clf, X_train, y_train)

True
	precision:	0.04 (+/- 0.00)
	recall:		0.87 (+/- 0.02)
	f1:		0.08 (+/- 0.00)

False
	precision:	0.99 (+/- 0.00)
	recall:		0.44 (+/- 0.03)
	f1:		0.61 (+/- 0.02)


### Заміняємо попередню фічу на індикатор, чи частота біграми '<слово> <крапка>' більша, ніж частота біграми '<слово> <наступне слово>'. Результат особливо не змінився :(

In [227]:
def bigrams_features_2(tok):
    feats = {}
    next_tok = tok.nbor(1) if tok.i <= len(tok.doc) - 2 else None
    freq = bigram_freq(tok.text, next_tok.text) if next_tok else 0
    period_freq = bigram_freq(tok.text, '.') if next_tok else 0
    feats['period>no-period'] = int(period_freq > freq)
    
    return feats

In [228]:
clf = make_classifier(word_features, bigrams_features_2)
cross_validation_report(clf, X_train, y_train)

True
	precision:	0.86 (+/- 0.03)
	recall:		0.51 (+/- 0.05)
	f1:		0.64 (+/- 0.03)

False
	precision:	0.99 (+/- 0.00)
	recall:		1.00 (+/- 0.00)
	f1:		0.99 (+/- 0.00)


### Кросвалідуємо та тестуємо фінальний класифікатор

In [229]:
clf = make_classifier(word_features, pos_features, dep_features, dep_children_features, bigrams_features_2)
cross_validation_report(clf, X_train, y_train)

True
	precision:	0.83 (+/- 0.03)
	recall:		0.58 (+/- 0.05)
	f1:		0.68 (+/- 0.03)

False
	precision:	0.99 (+/- 0.00)
	recall:		1.00 (+/- 0.00)
	f1:		0.99 (+/- 0.00)


In [230]:
clf.fit(X_train, y_train)
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

       False       0.99      0.99      0.99      4542
        True       0.75      0.59      0.66       155

    accuracy                           0.98      4697
   macro avg       0.87      0.79      0.82      4697
weighted avg       0.98      0.98      0.98      4697



## спостереження і висновки:
* довго мучився з біграмами, але покращення якості не вдалось досягти :(
  * можливо треба було збирати ще триграми;
  * також хотілося б зібрати н-грами частин мови, але в через phrasefinder.io це не можна зробити (принаймні я не знайшов), а з сирими гуглівськими н-грамами не було сил розбиратись;
  * також не знайшов як у phrasefinder шукати н-грами типу "<S\> word", а це могло б бути корисно;
  * але загалом phrasefinder - крутий проект, особливо враховуючи, що це one-man's project.
* перша ітерація класифікатору показала непоганий результат, але суттєво покращити його виявилось важкою задачею;
* можливо потрібно було підібрати інший корпус, наприклад корпус повідомлень в інтернеті чи щось подібне, який був би більш актуальний для задачі;
* також датасет та кількість фіч виявились досить великими, тому потрібно було відносно довго чекати поки класифікатор натренується.